In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import seaborn as sns
import pandas as pd
import numpy as np

import matplotlib.pyplot as mp
%matplotlib inline

col_acq = ['LoanID','Channel','SellerName','OrInterestRate','OrUnpaidPrinc','OrLoanTerm',
        'OrDate','FirstPayment','OrLTV','OrCLTV','NumBorrow','DTIRat','CreditScore',
        'FTHomeBuyer','LoanPurpose','PropertyType','NumUnits','OccStatus','PropertyState',
        'Zip','MortInsPerc','ProductType','CoCreditScore','MortInsType','RelMortInd'];

col_per = ['LoanID','MonthRep','Servicer','CurrInterestRate','CAUPB','LoanAge','MonthsToMaturity',
          'AdMonthsToMaturity','MaturityDate','MSA','CLDS','ModFlag','ZeroBalCode','ZeroBalDate',
          'LastInstallDate','ForeclosureDate','DispositionDate','PPRC','AssetRecCost','MHRC',
          'ATFHP','NetSaleProceeds','CreditEnhProceeds','RPMWP','OFP','NIBUPB','PFUPB','RMWPF',
          'FPWA','ServicingIndicator'];

from sklearn import preprocessing

# Choose datasets to be analyzed

In [ ]:
x= ['2001','2002',,'2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
y= ['Q1', 'Q2', 'Q3', 'Q4']

In [ ]:
z=[]
for i in x:
    for j in y:
        temp = i+j
        z.append(temp)

print(z)

In [ ]:
df_acq=pd.DataFrame()

for year_quarter in z:
    temp = pd.read_csv(f'/Users/tp/Desktop/FannieMae/Acquisition_All/Acquisition_{year_quarter}.txt', sep='|', names=col_acq, index_col= False)
    df_acq = pd.concat([df_acq, temp], axis=0)
    
df_acq.shape

In [ ]:
class_ification= pd.read_csv('/Users/tp/Desktop/FannieMae/classification.csv', sep=';', names=['PropertyState','Avg_Foreclosure_Time', 'Recourse', 'Judicial' ])

df_acq = pd.merge(df_acq, class_ification, how='inner', on='PropertyState')

In [ ]:
df_acq= df_acq.drop(['SellerName','OrDate','FirstPayment','PropertyState', 'Zip', 'CoCreditScore', ], axis=1)

In [ ]:

df_per=pd.DataFrame()

for year_quarter in z:
    temp = pd.read_csv(f'/Users/tp/Desktop/FannieMae/Performance_All/Performance_{year_quarter}.txt', sep='|', names= col_per, index_col= False, low_memory= False)
    df_per = pd.concat([df_per, temp], axis=0)

df_per.shape


In [ ]:
temp

In [ ]:
df_per.drop_duplicates(subset='LoanID', keep='last', inplace=True)
df = pd.merge(df_acq, df_per, on='LoanID', how='inner')

df.rename(index=str, columns={"ForeclosureDate": 'Default'}, inplace=True)

In [ ]:
df['Default'].fillna(0, inplace=True)
df.loc[df['Default'] != 0, 'Default'] = 1

df['Default'] = df['Default'].astype(int)

In [ ]:
#df['OrDateMonth'] = df['OrDate'].apply(lambda x: x.split('/')[0].strip()).astype(int)
#df['OrDateYear'] = df['OrDate'].apply(lambda x: x.split('/')[1].strip()).astype(int)

#df['FirstMonth'] = df['FirstPayment'].apply(lambda x: x.split('/')[0].strip()).astype(int)
#df['FirstYear'] = df['FirstPayment'].apply(lambda x: x.split('/')[1].strip()).astype(int)

#df.drop(['OrDate','FirstPayment'], axis=1, inplace=True)

In [ ]:

df.drop(['MortInsPerc','MortInsType','ProductType','LoanID'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df.groupby('Default').mean().transpose()

In [ ]:
for i in df.columns[df.dtypes == 'object']:
    a = (df.groupby(i).mean())
    print (pd.DataFrame(a).transpose())

In [ ]:
df.columns[df.dtypes=='object']

In [ ]:
pd.crosstab(df.Recourse, df.Judicial).plot(kind='bar', stacked = True)

In [ ]:
columns = df.columns[df.isnull().any()]
nan_cols = df[columns]

df.drop(nan_cols.columns, axis=1, inplace=True)

cat = df.select_dtypes(include=['object'])
num = df.drop(cat.columns, axis=1)

data = pd.DataFrame()
for i in cat.columns:
    tmp = pd.get_dummies(cat[i], prefix = i, drop_first=True)
    data = pd.concat([data, tmp], axis=1)

df = pd.concat([num,data,nan_cols], axis=1).reset_index(drop=True)


In [ ]:
columns = df.columns[df.isnull().any()]
for column in columns:
    col_mean = df.loc[df[column].notnull(), column].values.mean()
    df.loc[df[column].isnull(), column] = col_mean

In [ ]:
float_y = df.columns[df.dtypes=='float64']
for i in float_y:
    df[i].astype(int)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X = df.loc[:, df.columns != 'Default']

y= df.loc[:, df.columns == 'Default']

In [ ]:
os = SMOTE(random_state=0)

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [ ]:
print(X_train.shape)
print(type(X_train))
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
os_X, os_y =  os.fit_sample(X_train,y_train)

In [ ]:
os_X

In [ ]:
data_final_vars=df.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_X, os_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
columns = df.columns.values.tolist()

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
logre = LogisticRegression()

In [ ]:
rfe = RFE(logre, 10)

In [ ]:
rfe.fit(os_X, os_y.values.ravel())

In [ ]:
supp = rfe.support_

In [ ]:
to_be_used = pd.concat([pd.DataFrame(df.columns), pd.DataFrame(rfe.support_)], axis=1, ignore_index= True)

In [ ]:
to_be_used

In [ ]:
tbu = to_be_used[to_be_used[1]==True][0].tolist()

In [ ]:
tbu

In [ ]:
tbu.remove('Default')
tbu

In [ ]:
print(rfe.ranking_)

In [ ]:
X = os_X[tbu]

In [ ]:
y= os_y['Default']

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())